# Generate Passwords for Cracking Lab
In this lab, participants will attempt to crack password in several hash formats:
- MS Word
- Zip
- PDF
- NTLM (Active Directory)

The goal is to crack passwords for more critical documents or accounts. The more critical the document or account, the more points that are awarded for cracking it. Some accounts/documents will be difficult to crack without planning or research: brute force will not work.

## Documents


## NTLM Passwords
There are several types of passwords that have been generated for these exercises:
- Simple
- Complex
- Random
- Custom
- Password (based on "password")
- Keyboard (based on keyboard patterns)

The probability of the password for an account coming from any of these lists is based on the account criticality. Each account criticality type has a different probability distribution for password types. For example, critical accounts are less likely to have simple passwords, and more likely to have random passwords.

## Password Lengths
When it comes to random passwords, we choose a length between 5 and 17 characters. This means passwords tend to be short and crackable. It's not realistic but it allows for shorter excercises with some passwords still being in the "impossible" range.

# Load Password Lists

In [1]:
# Load our dependencies
import string
import random
import csv
import os
import subprocess

In [2]:
# Global Variables
word_dir="/home/ec2-user/hashcat/words"

password_types = ['simple', 'password', 'keyboard', 'complex', 'custom', 'random']
password_weights = {
    'low' : [60, 10, 10, 10, 5, 5],
    'medium' : [40, 10, 10, 25, 5, 10],
    'high' : [25, 5, 5, 30, 10, 25],
    'critical' : [5, 1, 1, 40, 25, 28]
}

accounts_to_generate = 5000

wordlist = {}

## Random Passwords
Depending on the length, these can be hard to crack. Humans tend not to use long random passwords, but we should include some.

In [3]:
characters = string.ascii_letters + string.digits + string.punctuation

wordlist['random'] = []

printables = len(characters)
for pcount in range(10000):
    password = ''.join(random.choice(characters) for i in range(random.choice(range(5,17))))
    wordlist['random'].append(password)

## Simple Passwords

These were generated with the following commands:
```
.\hashcat.exe --force --stdout --custom-charset1='2' --custom-charset2='0' --custom-charset3='012' \
    -a 6 ..\words\dictionary\english.txt '?1?2?3?d?s' > ..\words\datewords.txt 
.\hashcat.exe --force --stdout -a 0 -j 'c' ..\words\datewords.txt > simplewords.txt
```

In [4]:
# Load our list of Simple Passwords
fd = open(word_dir+'/simplewords.txt', 'r')
wordlist['simple'] = list(map(str.strip, fd.readlines()))
fd.close()

## Custom Passwords
This is a based on a custom list of words and made more complex through hashcat rules

In [23]:
fd = open(word_dir+'/customwords2.txt', 'r', encoding='ISO-8859-1')
wordlist['custom'] = list(map(str.strip, fd.readlines()))
fd.close()

## Keyboard Pattern Passwords
This is compiled from several existing lists of known keyboard patterns

```./hashcat.bin --force --stdout -r rules/dive.rule words/generated/KeyboardCombo7.dic```

In [16]:
key_count = 63941069 # number of lines in crackstation.txt
key_lines = random.sample(range(key_count), k=10000)
key_lines.sort()

fd = open(word_dir+'/keyboardwords.txt', 'r', encoding='ISO-8859-1')
line_num = 0
key_num = 0

wordlist['keyboard'] = []
## This will take about a 10 minute to run
while (key_num < 10000):
    line = fd.readline().strip()
    line_num += 1
    if line_num == crack_lines[key_num]:
#        print(f"{line_num} : {key_num} : {line}")
        key_num += 1
        wordlist['keyboard'].append(line)

fd.close()

## Based on Password
I have pregenerated a file that contains every variant of the word password. These are all based on character subsitutions and are all 8 characters long. However, there are other shorted common lists. To make our list we want to include some of these straight but also want to run them through some rules.
```
.\hashcat.exe --force --stdout -a 0 -r rules\\best64.rule ..\words\based\password.txt > password.txt
```

In [22]:
fd = open(word_dir+'/password.txt', 'r')
wordlist['password'] = list(map(str.strip, fd.readlines()))
fd.close()

## Complex Passwords
Complex passwords are those that are based on one or more words but have rules applied to them. They are not random but are based on harder to predict patterns. We will generate these using large wordlists, that already include plain and complex passwords, as well as rules

In [8]:
# Load a hashcat rule set into a list, so that we can randomly select a rule to use later
fd = open('/home/ec2-user/opt/password_cracking_rules/OneRuleToRuleThemAll.rule', 'r')
complex_rules = list(map(str.strip, fd.readlines()))
fd.close()

In [9]:
# Load a wordlist to combine with our rules to help generate complex passwords
## We are using crackstation, which is 15GB and takes a LONG time to load
crack_count = 63941069 # number of lines in crackstation.txt
crack_lines = random.sample(range(crack_count), k=10000)
crack_lines.sort()

fd = open('/home/ec2-user/opt/crackstation-human-only.txt', 'r', encoding='ISO-8859-1')
line_num = 0
crack_num = 0

complex_list = []
## This will take about a 1 minute to run
while (crack_num < 10000):
    line = fd.readline().strip()
    line_num += 1
    if line_num == crack_lines[crack_num]:
#        print(f"{line_num} : {crack_num} : {line}")
        crack_num += 1
        complex_list.append(line)

fd.close()


## Generate Accounts
Accounts have several important attributes:
- Account Name
- Department
- Criticality
- Password

NTLM accounts will be formated as `firstname.lastname`. I have made an effort to include a variety of names but they are probably still heavily eurocentric. Departments come from a CSV file and criticality is based on the department for randomly generated accounts. A seperate list of custom accounts with manually assigned criticality and passwords will be generated separately.

In [10]:
# Number of accounts to generate
# A list of accounts, will contain {'username', 'department', 'criticality', 'password', 'hash'}
# Generate usernames
fd = open(word_dir+'/first_names', 'r')
first_names = list(map(str.strip, fd.readlines()))
fd.close()

fd = open(word_dir+'/last_names', 'r')
last_names = list(map(str.strip, fd.readlines()))
fd.close()

# Load department list
fd = open('/home/ec2-user/opt/passlab/teach/departments.csv', 'r')
departments = list(csv.DictReader(fd))
fd.close()

In [11]:
# Generate Accounts and assign attributes
accounts = []

for i in range(accounts_to_generate):
    account_name = random.choice(first_names) + '.' + random.choice(last_names)
    dept = random.choice(departments)
    accounts.append({'account_name': account_name, 
                   'department': dept['department'], 
                   'criticality': dept['criticality']
                   })

## Assign Passwords to Accounts
For each account, we will assign or generate a password from one of our password lists. The probability of an account being assigned a simple, complex, random, custom, or other password type is based on a weighted list for the criticality of that account. For example, critical accounts are more likely to have random, custom, or complex passwords. Low criticality accounts are more likley to be assigned simple or pattern-based password. 

In [34]:
# Password Generation Function
def generate_password(type):
    if type=='complex':
        # These cannot be pre-generated from a list and are generated in real-time
        
        ## Select a rule from the rule list, and a word from the word list
        rule = random.choice(complex_rules)
        word = random.choice(complex_list)
        ## Then call hashcat in a shell to generate the password
        cmd = f'echo "{word}" | /home/ec2-user/hashcat/hashcat.bin --force --stdout -j "{rule}"'
        ## Return the password
        cmd_stream = os.popen(cmd)
        return cmd_stream.read().strip()
    else:
        return random.choice(wordlist[type]).strip()

In [35]:
for account in accounts:
    weights = password_weights[account['criticality']]
    password_type = random.choices(password_types, weights=weights)[0]
    account['password'] = generate_password(password_type)

In [27]:
accounts

[{'account_name': 'jacintha.coppolino',
  'department': 'Executive Office',
  'criticality': 'high',
  'password': '71B9C75E0744DA82C0F433D83C875DFD\n'},
 {'account_name': 'griffin.panela',
  'department': 'Trading Floor',
  'criticality': 'high',
  'password': 'eoh>y3p07x_.'},
 {'account_name': 'geno.egar',
  'department': 'Trading Floor',
  'criticality': 'high',
  'password': 'osrahjr11\n'},
 {'account_name': 'joye.brightful',
  'department': 'Credit Cards',
  'criticality': 'low',
  'password': 'Abiogenesis2013:'},
 {'account_name': 'ilda.twitt',
  'department': 'Software Development',
  'criticality': 'medium',
  'password': '9`#swOrd'},
 {'account_name': 'maritsa.rosensteel',
  'department': 'Wealth Management',
  'criticality': 'medium',
  'password': 'jacey2w1q!@#'},
 {'account_name': 'mart.sanghez',
  'department': 'Wealth Management',
  'criticality': 'medium',
  'password': 'queer-eye-logo\n'},
 {'account_name': 'pavla.mcgaha',
  'department': 'Executive Office',
  'critical

In [31]:
with open('accounts.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['account_name', 'department', 'criticality', 'password'])
    writer.writeheader()
    writer.writerows(accounts)